# 7.10 Piping Systems - Steam

40 bar steam, 450 degrees Celsius flows though a 120 m long horizontal 150mm schedule 80 pipe at a rate of
40000 kg/hr. 

There are three 90 degree weld elbows with rc=1.5, 
1 fully open class 600 150mm x 100mm venturi class gate valve, one class 600 150 mm class y pattern globe valve with a seat diameter of 90% the inside pipe diameter (disc fully lifted).

Calculate the pressure drop through the system.

In [1]:
from math import *
from fluids.units import *
from thermo.units import Stream

m = 40000*u.kg/u.hr
T = 450*u.degC
P = 40*u.bar

steam = Stream('water', T=T, P=P, m=m)

rho = steam.rho
mu = steam.mu
print('Density = %s' %rho)
print('Viscosity = %s' %mu)

Density = 12.493325597325969 kilogram / meter ** 3
Viscosity = 2.655716090653184e-05 pascal * second


In [2]:
L = 120*u.m
NPS, D_pipe, Do_pipe, t = nearest_pipe(Do=150*u.mm, schedule='80')
Q = m/rho
v = Q/(pi/4*D_pipe**2)
Re = Reynolds(rho=rho, mu=mu, D=D_pipe, V=v)
fd = friction_factor(Re=Re, eD=0.0018*u.inch/D_pipe)
K_elbow = bend_rounded(Di=D_pipe, angle=90*u.degrees, fd=fd, bend_diameters=1.5)
K_friction = K_from_f(fd=fd, L=L, D=D_pipe)
K_globe_valve = K_globe_valve_Crane(D1=0.9*D_pipe, D2=D_pipe, fd=fd)
# Angle and inside diameter are taken from example 7.4,
K_gate_valve = K_gate_valve_Crane(D1=100*u.mm, D2=D_pipe, angle=13.115*u.degrees, fd=fd)

K_tot = K_gate_valve + K_globe_valve + 3*K_elbow + K_friction
dP = dP_from_K(K=K_tot, rho=rho, V=v)

print('Crane elbow term = 0.63; calculated = %s' % (3*K_elbow))
print('Crane globe valve term = 1.44; calculated = %s' % (K_globe_valve))
print('Crane gate valve term = 1.22; calculated = %s' % (K_gate_valve))
print('Crane friction term = 12.3; calculated = %s' % (K_friction))
print('Darcy friction factor = %s' %fd)
print('Pressure drop = %s' %dP.to(u.Pa))

Crane elbow term = 0.63; calculated = 0.6059244854416919 dimensionless
Crane globe valve term = 1.44; calculated = 7.9714825451402715 dimensionless
Crane gate valve term = 1.22; calculated = 1.1715334417093926 dimensionless
Crane friction term = 12.3; calculated = 12.522948325321595 dimensionless
Darcy friction factor = 0.015273822640783908 dimensionless
Pressure drop = 388771.6702516167 pascal


This compares terribly to the example, which calculates a pressure drop of 274800 Pa!

Two errors are apparent - their example uses a different coefficient (55) in the globe valve pressure drop equation than that shown in their appendix (340); and they re-use their prior calculated gate valve, despite the friction factor being different in this example.
This, plus their use of a constant 0.015 friction factor, explains the difference. 

The example below uses their calculated globe valve pressure drop and their friction factor. The result (268500 Pa) compares well with their calculation; the additional decimals and better physical properties explain the rest. 

In [3]:
L = 120*u.m
NPS, D_pipe, Do_pipe, t = nearest_pipe(Do=150*u.mm, schedule='80')
Q = m/rho
v = Q/(pi/4*D_pipe**2)
Re = Reynolds(rho=rho, mu=mu, D=D_pipe, V=v)
fd = 0.015
K_elbow = bend_rounded(Di=D_pipe, angle=90*u.degrees, fd=fd, bend_diameters=1.5)
K_friction = K_from_f(fd=fd, L=L, D=D_pipe)
K_globe_valve = 1.44
# Angle and inside diameter are taken from example 7.4,
K_gate_valve = K_gate_valve_Crane(D1=100*u.mm, D2=D_pipe, angle=13.115*u.degrees, fd=fd)

K_tot = K_gate_valve + K_globe_valve + 3*K_elbow + K_friction
dP = dP_from_K(K=K_tot, rho=rho, V=v)

print('Crane elbow term = 0.63; calculated = %s' % (3*K_elbow))
print('Crane globe valve term = 1.44; calculated = %s' % (K_globe_valve))
print('Crane gate valve term = 1.22; calculated = %s' % (K_gate_valve))
print('Crane friction term = 12.3; calculated = %s' % (K_friction))
print('Darcy friction factor = %s' %fd)
print('Pressure drop = %s' %dP.to(u.Pa))

Crane elbow term = 0.63; calculated = 0.5988647407128814 dimensionless
Crane globe valve term = 1.44; calculated = 1.44
Crane gate valve term = 1.22; calculated = 1.1614815235317342 dimensionless
Crane friction term = 12.3; calculated = 12.29844219732167 dimensionless
Darcy friction factor = 0.015
Pressure drop = 270542.3833530189 pascal
